# Amplicon processing tutorial

## Initial setup

### Setting our location, some variables, and getting data

In [28]:
# dir.create("~/amplicon-tutorial", showWarnings = FALSE)
# setwd("~/amplicon-tutorial/")
# # download and unpack
# # change into dir
# # setwd("~/

# ## for now while testing/building
# # full location
# # source_dir <- "/Users/mdlee4/Documents/NASA/GeneLab/GL4U/amplicon/GLDS-249-full-reads"

# # subset location
# source_dir <- "/Users/mdlee4/Documents/NASA/GeneLab/GL4U/amplicon/GLDS-249-subset"
# files_to_copy <- list.files(source_dir, full.names = TRUE)

# raw_reads_dir <- "Raw_Sequence_Data/"
# trimmed_and_filtered_reads_dir <- "Trimmed_and_Filtered_Sequence_Data/"
# fastqc_outputs_dir <- "FastQC_Outputs"
# final_outputs_dir <- "Final_Outputs"

# file.copy(files_to_copy, "./", overwrite = TRUE)

[1] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
[16] TRUE TRUE TRUE TRUE TRUE

In [43]:
dir.create("~/amplicon-tutorial2/", showWarnings = FALSE)
setwd("~/amplicon-tutorial2/")

In [52]:
raw_reads_dir <- "Raw_Sequence_Data/"
# trimmed_reads_dir <- "Trimmed_Sequence_Data"
trimmed_and_filtered_reads_dir <- "Trimmed_and_Filtered_Sequence_Data/"
fastqc_outputs_dir <- "FastQC_Outputs/"
final_outputs_dir <- "Final_Outputs/"

In [53]:
# sapply(c(raw_reads_dir, trimmed_reads_dir, filtered_reads_dir, fastqc_outputs_dir), dir.create)
dir.create(raw_reads_dir)
# dir.create(trimmed_reads_dir)
dir.create(trimmed_and_filtered_reads_dir)
dir.create(fastqc_outputs_dir)

Warning message in dir.create(raw_reads_dir):
“'Raw_Sequence_Data' already exists”
Warning message in dir.create(trimmed_and_filtered_reads_dir):
“'Trimmed_and_Filtered_Sequence_Data' already exists”
Warning message in dir.create(fastqc_outputs_dir):
“'FastQC_Outputs' already exists”


In [54]:
list.files()

[1] "FastQC_Outputs"                     "Raw_Sequence_Data"                 
[3] "Trimmed_and_Filtered_Sequence_Data"

In [55]:
## for now while testing/building
# full location
# source_dir <- "/Users/mdlee4/Documents/NASA/GeneLab/GL4U/amplicon/GLDS-249-full-reads"

# subset location
source_dir <- "/Users/mdlee4/Documents/NASA/GeneLab/GL4U/amplicon/GLDS-249-subset"
files_to_copy <- list.files(source_dir, full.names = TRUE)

file.copy(files_to_copy, raw_reads_dir, overwrite = TRUE)

[1] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
[16] TRUE TRUE TRUE TRUE TRUE

In [56]:
list.files(raw_reads_dir)

[1] "F10_R1_raw.fastq.gz" "F10_R2_raw.fastq.gz" "F3_R1_raw.fastq.gz" 
 [4] "F3_R2_raw.fastq.gz"  "F5_R1_raw.fastq.gz"  "F5_R2_raw.fastq.gz" 
 [7] "F8_R1_raw.fastq.gz"  "F8_R2_raw.fastq.gz"  "F9_R1_raw.fastq.gz" 
[10] "F9_R2_raw.fastq.gz"  "G10_R1_raw.fastq.gz" "G10_R2_raw.fastq.gz"
[13] "G3_R1_raw.fastq.gz"  "G3_R2_raw.fastq.gz"  "G5_R1_raw.fastq.gz" 
[16] "G5_R2_raw.fastq.gz"  "G8_R1_raw.fastq.gz"  "G8_R2_raw.fastq.gz" 
[19] "G9_R1_raw.fastq.gz"  "G9_R2_raw.fastq.gz"

In [63]:
# making a variable holding the unique sample names for use with various things
sample_names <- list.files(raw_reads_dir, pattern = "_R1_") %>% gsub(pattern = "_R1_raw.fastq.gz", replacement = "")

In [64]:
sample_names

[1] "F10" "F3"  "F5"  "F8"  "F9"  "G10" "G3"  "G5"  "G8"  "G9"

### Loading libraries

In [33]:
library(dada2)
library(tidyverse)

Loading required package: Rcpp

── Attaching packages ───────────────────────────────────────────────────────────────────────────────────────── tidyverse 1.3.2 ──
✔ ggplot2 3.4.1     ✔ purrr   1.0.1
✔ tibble  3.1.8     ✔ dplyr   1.1.0
✔ tidyr   1.3.0     ✔ stringr 1.5.0
✔ readr   2.1.4     ✔ forcats 1.0.0
── Conflicts ──────────────────────────────────────────────────────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


### Creating some variables to help with processing

In [65]:
# making an object that holds all forward read starting files
forward_raw_files <- list.files(raw_reads_dir, pattern = "*R1_raw.fastq.gz", full.names = TRUE)

# making an object that holds all reverse read starting file
reverse_raw_files <- list.files(raw_reads_dir, pattern = "*R2_raw.fastq.gz", full.names = TRUE)

In [66]:
forward_raw_files

[1] "Raw_Sequence_Data//F10_R1_raw.fastq.gz"
 [2] "Raw_Sequence_Data//F3_R1_raw.fastq.gz" 
 [3] "Raw_Sequence_Data//F5_R1_raw.fastq.gz" 
 [4] "Raw_Sequence_Data//F8_R1_raw.fastq.gz" 
 [5] "Raw_Sequence_Data//F9_R1_raw.fastq.gz" 
 [6] "Raw_Sequence_Data//G10_R1_raw.fastq.gz"
 [7] "Raw_Sequence_Data//G3_R1_raw.fastq.gz" 
 [8] "Raw_Sequence_Data//G5_R1_raw.fastq.gz" 
 [9] "Raw_Sequence_Data//G8_R1_raw.fastq.gz" 
[10] "Raw_Sequence_Data//G9_R1_raw.fastq.gz"

In [67]:
reverse_raw_files

[1] "Raw_Sequence_Data//F10_R2_raw.fastq.gz"
 [2] "Raw_Sequence_Data//F3_R2_raw.fastq.gz" 
 [3] "Raw_Sequence_Data//F5_R2_raw.fastq.gz" 
 [4] "Raw_Sequence_Data//F8_R2_raw.fastq.gz" 
 [5] "Raw_Sequence_Data//F9_R2_raw.fastq.gz" 
 [6] "Raw_Sequence_Data//G10_R2_raw.fastq.gz"
 [7] "Raw_Sequence_Data//G3_R2_raw.fastq.gz" 
 [8] "Raw_Sequence_Data//G5_R2_raw.fastq.gz" 
 [9] "Raw_Sequence_Data//G8_R2_raw.fastq.gz" 
[10] "Raw_Sequence_Data//G9_R2_raw.fastq.gz"

In [68]:
# making an object holding what will be the output trimmed/filtered forward files
forward_filtered_files <- paste0(trimmed_and_filtered_reads_dir, sample_names, "_R1_filtered.fastq.gz")

# making an object holding what will be the output trimmed/filtered reverse files
reverse_filtered_files <- paste0(trimmed_and_filtered_reads_dir, sample_names, "_R2_filtered.fastq.gz")

In [69]:
forward_filtered_files

[1] "Trimmed_and_Filtered_Sequence_Data/F10_R1_filtered.fastq.gz"
 [2] "Trimmed_and_Filtered_Sequence_Data/F3_R1_filtered.fastq.gz" 
 [3] "Trimmed_and_Filtered_Sequence_Data/F5_R1_filtered.fastq.gz" 
 [4] "Trimmed_and_Filtered_Sequence_Data/F8_R1_filtered.fastq.gz" 
 [5] "Trimmed_and_Filtered_Sequence_Data/F9_R1_filtered.fastq.gz" 
 [6] "Trimmed_and_Filtered_Sequence_Data/G10_R1_filtered.fastq.gz"
 [7] "Trimmed_and_Filtered_Sequence_Data/G3_R1_filtered.fastq.gz" 
 [8] "Trimmed_and_Filtered_Sequence_Data/G5_R1_filtered.fastq.gz" 
 [9] "Trimmed_and_Filtered_Sequence_Data/G8_R1_filtered.fastq.gz" 
[10] "Trimmed_and_Filtered_Sequence_Data/G9_R1_filtered.fastq.gz"

In [70]:
reverse_filtered_files

[1] "Trimmed_and_Filtered_Sequence_Data/F10_R2_filtered.fastq.gz"
 [2] "Trimmed_and_Filtered_Sequence_Data/F3_R2_filtered.fastq.gz" 
 [3] "Trimmed_and_Filtered_Sequence_Data/F5_R2_filtered.fastq.gz" 
 [4] "Trimmed_and_Filtered_Sequence_Data/F8_R2_filtered.fastq.gz" 
 [5] "Trimmed_and_Filtered_Sequence_Data/F9_R2_filtered.fastq.gz" 
 [6] "Trimmed_and_Filtered_Sequence_Data/G10_R2_filtered.fastq.gz"
 [7] "Trimmed_and_Filtered_Sequence_Data/G3_R2_filtered.fastq.gz" 
 [8] "Trimmed_and_Filtered_Sequence_Data/G5_R2_filtered.fastq.gz" 
 [9] "Trimmed_and_Filtered_Sequence_Data/G8_R2_filtered.fastq.gz" 
[10] "Trimmed_and_Filtered_Sequence_Data/G9_R2_filtered.fastq.gz"

## Quality screening of starting data

### FastQC on all

In [52]:
system2("fastqc", "-t 4", paste0(*.gz")

In [53]:
list.files()

[1] "F10_R1_raw_fastqc.html" "F10_R1_raw_fastqc.zip"  "F10_R1_raw.fastq.gz"   
 [4] "F10_R2_raw_fastqc.html" "F10_R2_raw_fastqc.zip"  "F10_R2_raw.fastq.gz"   
 [7] "F3_R1_raw_fastqc.html"  "F3_R1_raw_fastqc.zip"   "F3_R1_raw.fastq.gz"    
[10] "F3_R2_raw_fastqc.html"  "F3_R2_raw_fastqc.zip"   "F3_R2_raw.fastq.gz"    
[13] "F5_R1_raw_fastqc.html"  "F5_R1_raw_fastqc.zip"   "F5_R1_raw.fastq.gz"    
[16] "F5_R2_raw_fastqc.html"  "F5_R2_raw_fastqc.zip"   "F5_R2_raw.fastq.gz"    
[19] "F8_R1_raw_fastqc.html"  "F8_R1_raw_fastqc.zip"   "F8_R1_raw.fastq.gz"    
[22] "F8_R2_raw_fastqc.html"  "F8_R2_raw_fastqc.zip"   "F8_R2_raw.fastq.gz"    
[25] "F9_R1_raw_fastqc.html"  "F9_R1_raw_fastqc.zip"   "F9_R1_raw.fastq.gz"    
[28] "F9_R2_raw_fastqc.html"  "F9_R2_raw_fastqc.zip"   "F9_R2_raw.fastq.gz"    
[31] "G10_R1_raw_fastqc.html" "G10_R1_raw_fastqc.zip"  "G10_R1_raw.fastq.gz"   
[34] "G10_R2_raw_fastqc.html" "G10_R2_raw_fastqc.zip"  "G10_R2_raw.fastq.gz"   
[37] "G3_R1_raw_fastqc.html"  "G3_R1_raw_fastqc.zip"   "G3_R1_raw.fastq.gz"    
[40] "G3_R2_raw_fastqc.html"  "G3_R2_raw_fastqc.zip"   "G3_R2_raw.fastq.gz"    
[43] "G5_R1_raw_fastqc.html"  "G5_R1_raw_fastqc.zip"   "G5_R1_raw.fastq.gz"    
[46] "G5_R2_raw_fastqc.html"  "G5_R2_raw_fastqc.zip"   "G5_R2_raw.fastq.gz"    
[49] "G8_R1_raw_fastqc.html"  "G8_R1_raw_fastqc.zip"   "G8_R1_raw.fastq.gz"    
[52] "G8_R2_raw_fastqc.html"  "G8_R2_raw_fastqc.zip"   "G8_R2_raw.fastq.gz"    
[55] "G9_R1_raw_fastqc.html"  "G9_R1_raw_fastqc.zip"   "G9_R1_raw.fastq.gz"    
[58] "G9_R2_raw_fastqc.html"  "G9_R2_raw_fastqc.zip"   "G9_R2_raw.fastq.gz"

We can look at one of these by navigating to it on the left and double-clicking on the html file

In [55]:
# temp code to copy one over while i'm building
file.copy("G5_R1_raw_fastqc.html", "/Users/mdlee4/github/AstrobioMike.github.io/jupyter-notebooks/amplicon/")

[1] TRUE

Most of these not important for amplicon data, but let's think about the per base sequence content for a second...

- put in image here when finalizing things Mike

### MultiQC to combine them into one report

In [57]:
system("multiqc -n raw_multiqc . -o ")

We can look at this too by nagivating to it and double-clicking this html file:

In [58]:
# temp code to copy one over while i'm building
file.copy("raw_multiqc.html", "/Users/mdlee4/github/AstrobioMike.github.io/jupyter-notebooks/amplicon/")

[1] TRUE

In [114]:
list.files(pattern = "fastqc")

[1] "F10_R1_raw_fastqc.html" "F10_R1_raw_fastqc.zip"  "F10_R2_raw_fastqc.html"
 [4] "F10_R2_raw_fastqc.zip"  "F3_R1_raw_fastqc.html"  "F3_R1_raw_fastqc.zip"  
 [7] "F3_R2_raw_fastqc.html"  "F3_R2_raw_fastqc.zip"   "F5_R1_raw_fastqc.html" 
[10] "F5_R1_raw_fastqc.zip"   "F5_R2_raw_fastqc.html"  "F5_R2_raw_fastqc.zip"  
[13] "F8_R1_raw_fastqc.html"  "F8_R1_raw_fastqc.zip"   "F8_R2_raw_fastqc.html" 
[16] "F8_R2_raw_fastqc.zip"   "F9_R1_raw_fastqc.html"  "F9_R1_raw_fastqc.zip"  
[19] "F9_R2_raw_fastqc.html"  "F9_R2_raw_fastqc.zip"   "G10_R1_raw_fastqc.html"
[22] "G10_R1_raw_fastqc.zip"  "G10_R2_raw_fastqc.html" "G10_R2_raw_fastqc.zip" 
[25] "G3_R1_raw_fastqc.html"  "G3_R1_raw_fastqc.zip"   "G3_R2_raw_fastqc.html" 
[28] "G3_R2_raw_fastqc.zip"   "G5_R1_raw_fastqc.html"  "G5_R1_raw_fastqc.zip"  
[31] "G5_R2_raw_fastqc.html"  "G5_R2_raw_fastqc.zip"   "G8_R1_raw_fastqc.html" 
[34] "G8_R1_raw_fastqc.zip"   "G8_R2_raw_fastqc.html"  "G8_R2_raw_fastqc.zip"  
[37] "G9_R1_raw_fastqc.html"  "G9_R1_raw_fastqc.zip"   "G9_R2_raw_fastqc.html" 
[40] "G9_R2_raw_fastqc.zip"

In [115]:
# and removing regular fastqc outputs now that they are grouped by multiqc
list.files(pattern = "fastqc") %>% file.remove()

[1] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
[16] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
[31] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE

In [116]:
list.files()

[1] "F10_R1_filtered.fastq.gz" "F10_R1_raw.fastq.gz"     
 [3] "F10_R2_filtered.fastq.gz" "F10_R2_raw.fastq.gz"     
 [5] "F3_R1_filtered.fastq.gz"  "F3_R1_raw.fastq.gz"      
 [7] "F3_R2_filtered.fastq.gz"  "F3_R2_raw.fastq.gz"      
 [9] "F5_R1_filtered.fastq.gz"  "F5_R1_raw.fastq.gz"      
[11] "F5_R2_filtered.fastq.gz"  "F5_R2_raw.fastq.gz"      
[13] "F8_R1_filtered.fastq.gz"  "F8_R1_raw.fastq.gz"      
[15] "F8_R2_filtered.fastq.gz"  "F8_R2_raw.fastq.gz"      
[17] "F9_R1_filtered.fastq.gz"  "F9_R1_raw.fastq.gz"      
[19] "F9_R2_filtered.fastq.gz"  "F9_R2_raw.fastq.gz"      
[21] "G10_R1_filtered.fastq.gz" "G10_R1_raw.fastq.gz"     
[23] "G10_R2_filtered.fastq.gz" "G10_R2_raw.fastq.gz"     
[25] "G3_R1_filtered.fastq.gz"  "G3_R1_raw.fastq.gz"      
[27] "G3_R2_filtered.fastq.gz"  "G3_R2_raw.fastq.gz"      
[29] "G5_R1_filtered.fastq.gz"  "G5_R1_raw.fastq.gz"      
[31] "G5_R2_filtered.fastq.gz"  "G5_R2_raw.fastq.gz"      
[33] "G8_R1_filtered.fastq.gz"  "G8_R1_raw.fastq.gz"      
[35] "G8_R2_filtered.fastq.gz"  "G8_R2_raw.fastq.gz"      
[37] "G9_R1_filtered.fastq.gz"  "G9_R1_raw.fastq.gz"      
[39] "G9_R2_filtered.fastq.gz"  "G9_R2_raw.fastq.gz"      
[41] "raw_multiqc_data"         "raw_multiqc.html"

## Checking on settings to remove primers

It is imperative that we properly remove the primers. We can try trimming the primers with dada2's quality trimming/filtering program. But before we run it on everything, we're going to closely look at and test things on one sample – looking at the sequences before and after so we can visibly check the primers are indeed being removed.

These are the primers for this dataset, and the IUPAC degenerate-base codes.

```
f primer: GTGYCAGCMGCCGCGGTAA
r primer: GGACTACNVGGGTWTCTAAT

Y = C/T  
M = A/C  
N = A/T/G/C  
V = A/C/G  
W = A/T  
```

### Ensuring we can spot the primers

In [72]:
# establishing a connect with the F10 forward read file
incon <- gzcon(file("F10_R1_raw.fastq.gz", open = "rb"))

In [73]:
# this reads in the first 8 lines, with each set of 4 lines holding one fastq entry
stuff <- readLines(incon, 8)

In [74]:
# here is how we can just get the sequences for the first 2 entries
stuff[c(2,6)]

[1] "GTGCCAGCAGCCGCGGTAATACGGAGGATGCGAGCGTTATCCGGATTTAATGGGTTTAAAGGGTGCGTAGGTGGTTAATTAAGTCAGCGGTGAAAGTATGTGGCTCAACCATAAAATTGCCGTTGAAACTGGTTGACTTGAGTATAGTAGAGG"
[2] "GTGCCAGCCGCCGCGGTAATACGTAGGGGGCAAGCGTTATCCGGATTTACTGGGAGTAAAGGGAGCGTAGACGGCCAGACAAGTCTGAAGTGAAAATTCAGCGCTTAACTTTGGAAGTGCTTTGGAAACTGCCGGGATAGAGTGCAGGAGGGG"

They each start exactly with the forward primer sequence right up front, which isn't always the case (the asterisks are over the degenerate bases):

```
                           *    *
forward primer:         GTGYCAGCMGCCGCGGTAA
forward read 1 start:   GTGCCAGCAGCCGCGGTAA
forward read 2 start:   GTGCCAGCCGCCGCGGTAA
```

Let's look at a couple reverse reads:

In [75]:
# establishing a connection
incon <- gzcon(file("F10_R2_raw.fastq.gz", open = "rb"))

# storing the first 8 lines in a variable
stuff_rev <- readLines(incon, 8)

In [77]:
# and looking at the first 2 sequences
stuff_rev[c(2,6)]

[1] "GGACTACTAGGGTTTCTAATCCTGTTTGATCCCCACGCTTTCGTGCATCAGTGTCAGTTATAGAAAAGTAAGCAGCCTTCGCAAACGGAGTTCAGCGAGATATCTAAGCATTTCACCGCTANACNANGNNTTCCGCCTACCTCAAATAAACTC"
[2] "GGACTACCCGGGTTTCTAATCCTTTTTGCTCCCCACGCCTTCGAGCCTCAACGTCAGTTGCAGTCCAGCAGGACGCCTTCGCCACTGGTGTTCCTCCTAATATCTACGCATTTCACCGCTANACTANGAATTCCGCCTGCCCCTCCTGCACTC"

They each start exactly with the reverse primer sequence right up front:

```
                                **    *
reverse primer:          GGACTACNVGGGTWTCTAAT
reverse read 1 start:    GGACTACTAGGGTTTCTAAT
reverse read 2 start:    GGACTACCCGGGTTTCTAAT
```

### Doing a test trimming where we specify to cut these off
The forward primer is 19 bases, the reverse is 20. We can pass these values to the `trimLeft` argument of dada2's `filterAndTrim()` function:

In [66]:
filterAndTrim(fwd = "F10_R1_raw.fastq.gz", 
              rev = "F10_R2_raw.fastq.gz", 
              filt = "F10_R1_filtered.fastq.gz",
              filt.rev = "F10_R2_filtered.fastq.gz", 
              trimLeft = c(19, 20))

### Ensuring those settings successfully removed the primers
Now we are going to peek at the output trimmed files to make sure we cut off the primers, doing the same things we did above to read in part of the file and then just look at the first 2 sequences of the forward and reverse reads:

In [79]:
# establishing a connection and storing the first 8 lines into a file in one line now
filt_stuff <- gzcon(file("F10_R1_filtered.fastq.gz", open = "rb")) %>% readLines(8)
    # reminder that this is the same as writing things nested this way
# filt_stuff <- readLines(gzcon(file("F10_R1_filtered.fastq.gz", open = "rb")), 8)

In [81]:
filt_stuff[c(2,6)]

[1] "TACGGAGGATGCGAGCGTTATCCGGATTTAATGGGTTTAAAGGGTGCGTAGGTGGTTAATTAAGTCAGCGGTGAAAGTATGTGGCTCAACCATAAAATTGCCGTTGAAACTGGTTGACTTGAGTATAGTAGAGG"
[2] "TACGTAGGGGGCAAGCGTTATCCGGATTTACTGGGAGTAAAGGGAGCGTAGACGGCCAGACAAGTCTGAAGTGAAAATTCAGCGCTTAACTTTGGAAGTGCTTTGGAAACTGCCGGGATAGAGTGCAGGAGGGG"

These previously started:

```
                            *    *
forward primer:          GTGYCAGCMGCCGCGGTAA
original fwd read 1:     GTGCCAGCAGCCGCGGTAA   TACGGAGGAT
original fwd read 2:     GTGCCAGCCGCCGCGGTAA   TACGTAGGGG
```

They each now begin right after the forward primer 👍

In [84]:
filt_stuff_rev <- gzcon(file("F10_R2_filtered.fastq.gz", open = "rb")) %>% readLines(8)

In [85]:
filt_stuff_rev[c(2,6)]

[1] "CCTGTTTGATCCCCACGCTTTCGTGCATCAGTGTCAGTTATAGAAAAGTAAGCAGCCTTCGCAAACGGAGTTCAGCGAGATATCTAAGCATTTCACCGCTA"
[2] "CCTTTTTGCTCCCCACGCCTTCGAGCCTCAACGTCAGTTGCAGTCCAGCAGGACGCCTTCGCCACTGGTGTTCCTCCTAATATCTACGCATTTCACCGCTA"

These previously started:

```
                                **    *
reverse primer:          GGACTACNVGGGTWTCTAAT
original rev read 1:     GGACTACTAGGGTTTCTAAT  CCTGTTTGAT
original rev read 2:     GGACTACCCGGGTTTCTAAT  CCTTTTTGCT
```

They each now begin right after the reverse primer 👍

So with that confirmation (or looking at some more samples if wanted), we can be fairly confident in using that `trimLeft` argument for all our samples to remove the primers (since these were all prepared and sequenced together the same way).

Now just removing those test output files so we know for sure we run everything the same way when we do all of them:

In [104]:
file.remove("F10_R1_filtered.fastq.gz", "F10_R2_filtered.fastq.gz")

[1] TRUE TRUE

## Processing with dada2

### Quality trimming/filtering (including removing primers)

In [106]:
filtered_out <- filterAndTrim(fwd = forward_raw_files, 
                              rev = reverse_raw_files, 
                              filt = forward_filtered_files, 
                              filt.rev = reverse_filtered_files, 
                              trimLeft = c(19, 20), 
                              maxEE = c(1,1),
                              multithread = 4)

**I'm not sure which of these formats i like better yet**

<div class="alert alert-block alert-info">
<b>Function breakdown</b>
<br>
</div>

- `filterAndTrim()` - primary function
    - `fwd = `      - where we provide the object holding all the forward read input files
    - `rev = `      - where we provide the object holding all the reverse read input files
    - `filt = `     - where we provide the object holding what will be the output forward read files
    - `filt.rev = ` - where we provide the object holding what will be the output reverse read files
    - `trimLeft = ` - how many bases we want to have trimmed off the left side of the reads (providing them as a vector like this with two numbers means the first will used for the forward reads and the second for the reverse reads)
    - `maxEE = `    - maximum "expected error" to allow for the forward and reverse reads (similar to above; you can read more about "expected error" [here](https://www.drive5.com/usearch/manual/exp_errs.html) and in its original publication [here](https://academic.oup.com/bioinformatics/article/31/21/3476/194979))
    
<div class="alert alert-block alert-info">
<b><hr></b>
</div>


<div class="alert alert-block alert-info">
<b>Function breakdown</b>
<br>

- `filterAndTrim()` - primary function
    - `fwd = `      - where we provide the object holding all the forward read input files
    - `rev = `      - where we provide the object holding all the reverse read input files
    - `filt = `     - where we provide the object holding what will be the output forward read files
    - `filt.rev = ` - where we provide the object holding what will be the output reverse read files
    - `trimLeft = ` - how many bases we want to have trimmed off the left side of the reads (providing them as a vector like this with two numbers means the first will used for the forward reads and the second for the reverse reads)
    - `maxEE = `    - maximum "expected error" to allow for the forward and reverse reads (similar to above; you can read more about "expected error" [here](https://www.drive5.com/usearch/manual/exp_errs.html) and in its original publication [here](https://academic.oup.com/bioinformatics/article/31/21/3476/194979))

</div>


**Switch back to amplicon-QC.ipynb in order to run fastqc/multiqc and look at these (if i decide to do that at the command line)**

### Generate error model of data

In [117]:
# took about 45 seconds with subset dataset on local with 4 cpus
err_forward_reads <- learnErrors(forward_filtered_files, multithread = 4)
err_reverse_reads <- learnErrors(reverse_filtered_files, multithread = 4)

35427743 total bases in 264532 reads from 10 samples will be used for learning the error rates.
35167163 total bases in 264532 reads from 10 samples will be used for learning the error rates.


### Inferring sequences

In [118]:
# took about 20 seconds with subset dataset on local with 4 cpus
forward_seqs <- dada(forward_filtered_files, err = err_forward_reads, pool = "pseudo", multithread = 4)
reverse_seqs <- dada(reverse_filtered_files, err = err_reverse_reads, pool = "pseudo", multithread = 4)

Sample 1 - 26560 reads in 5943 unique sequences.
Sample 2 - 27920 reads in 5622 unique sequences.
Sample 3 - 28023 reads in 5632 unique sequences.
Sample 4 - 26796 reads in 5478 unique sequences.
Sample 5 - 24666 reads in 5149 unique sequences.
Sample 6 - 23554 reads in 4555 unique sequences.
Sample 7 - 27674 reads in 5859 unique sequences.
Sample 8 - 28505 reads in 5894 unique sequences.
Sample 9 - 28276 reads in 5248 unique sequences.
Sample 10 - 22558 reads in 4454 unique sequences.

   selfConsist step 2Sample 1 - 26560 reads in 5985 unique sequences.
Sample 2 - 27920 reads in 5528 unique sequences.
Sample 3 - 28023 reads in 5558 unique sequences.
Sample 4 - 26796 reads in 5454 unique sequences.
Sample 5 - 24666 reads in 5174 unique sequences.
Sample 6 - 23554 reads in 4375 unique sequences.
Sample 7 - 27674 reads in 5730 unique sequences.
Sample 8 - 28505 reads in 5747 unique sequences.
Sample 9 - 28276 reads in 5235 unique sequences.
Sample 10 - 22558 reads in 4381 unique sequenc

### Merging forward and reverse reads

In [119]:
merged_amplicons <- mergePairs(dadaF = forward_seqs, derepF = forward_filtered_files, 
                               dadaR = reverse_seqs, derepR = reverse_filtered_files)

### Generating sequence table with counts per sample

In [120]:
seqtab <- makeSequenceTable(merged_amplicons)

### Removing putative chimeras

In [121]:
seqtab.nochim <- removeBimeraDenovo(seqtab, multithread = 4)

In [122]:
dim(seqtab)

[1]  10 504

In [123]:
dim(seqtab.nochim)

[1] 10 85

In [124]:
sum(seqtab.nochim) / sum(seqtab) * 100

[1] 95.76756

We retained 96% of the initial sequences.

### Assigning taxonomy

In [125]:
# loading library used for taxonomy assignment
library(DECIPHER)

Loading required package: Biostrings

Loading required package: BiocGenerics

Loading required package: parallel


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB


The following objects are masked from ‘package:dplyr’:

    combine, intersect, setdiff, union


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, basename, cbind, colnames,
    dirname, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
    grepl, intersect, is.unsorted, lapply, Map, mapply, match, mget,
    order, paste, pmax, pmax.int, pmin, pmin.int, Position, rank,
    rbind, Reduce, rownames, sapply, setdiff, sort, table, tapply,
    union, unique, unsplit, whi

In [126]:
# creating the type of object needed
dna <- DNAStringSet(getSequences(seqtab.nochim))

In [127]:
# downloading reference PUT THIS SOMEWHERE, MIKE, MAYBE ON THE SYSTEM, OR MAYBE JUST A BACKUP DOWNLOADABLE FROM FIGSHARE
download.file(url = "http://www2.decipher.codes/Classification/TrainingSets/SILVA_SSU_r138_2019.RData", destfile = "SILVA_SSU_r138_2019.RData")

In [128]:
# loading reference
load("SILVA_SSU_r138_2019.RData")

In [129]:
# took about 60 seconds with subset dataset on local with 4 cpus
# classifying sequences
tax_info <- IdTaxa(dna, trainingSet = trainingSet, strand = "both", processors = 4)


Time difference of 48.28 secs



In [130]:
tax_info

  A test set of class 'Taxa' with length 85
     confidence taxon
 [1]        97% Root; Bacteria; Bacteroidota; Bacteroidia; Bacteroidales; Tan...
 [2]        65% Root; Bacteria; Firmicutes; Clostridia; Lachnospirales; Lachn...
 [3]        96% Root; Bacteria; Firmicutes; Bacilli; Erysipelotrichales; Erys...
 [4]        69% Root; Bacteria; Firmicutes; Clostridia; Oscillospirales; Rumi...
 [5]        67% Root; Bacteria; Firmicutes; Clostridia; Lachnospirales; Lachn...
 ...        ... ...
[81]        61% Root; Bacteria; Proteobacteria; Gammaproteobacteria; Oceanosp...
[82]        64% Root; Bacteria; Firmicutes; Bacilli; Paenibacillales; Paeniba...
[83]        74% Root; Bacteria; Firmicutes; Clostridia; Lachnospirales; Lachn...
[84]        74% Root; Bacteria; Bacteroidota; Bacteroidia; unclassified_Bacte...
[85]        90% Root; Bacteria; Firmicutes; Clostridia; Lachnospirales; Lachn...

### Generating and writing standard outputs

In [131]:
# giving sequences more manageable names
asv_seqs <- colnames(seqtab.nochim)
asv_headers <- vector(dim(seqtab.nochim)[2], mode = "character")

for (i in 1:dim(seqtab.nochim)[2]) {
    asv_headers[i] <- paste(">ASV", i, sep = "_")
}

In [132]:
# making then writing out a fasta of final ASV sequences
asv_fasta <- c(rbind(asv_headers, asv_seqs))
write(asv_fasta, "ASVs.fasta")

In [133]:
# making and writing out a count table
asv_tab <- t(seqtab.nochim)
row.names(asv_tab) <- sub(">", "", asv_headers)

write.table(asv_tab, "ASV_counts.tsv", sep = "\t", quote = F, col.names = NA)

In [134]:
# making and writing out a table of taxonomy, with any unclassified as "NA"
ranks <- c("domain", "phylum", "class", "order", "family", "genus", "species")

tax_tab <- t(sapply(tax_info, function(x) {
    m <- match(ranks, x$rank)
    taxa <- x$taxon[m]
    taxa[startsWith(taxa, "unclassified_")] <- NA
    taxa
}))

colnames(tax_tab) <- ranks
rownames(tax_tab) <- gsub(pattern = ">", replacement = "", x = asv_headers)

write.table(tax_tab, "ASV_taxonomy.tsv", sep = "\t", quote = F, col.names = NA)

In [135]:
tax_tab

,domain,phylum,class,order,family,genus,species
ASV_1,Bacteria,Bacteroidota,Bacteroidia,Bacteroidales,Tannerellaceae,Parabacteroides,NA
ASV_2,Bacteria,Firmicutes,Clostridia,Lachnospirales,Lachnospiraceae,Lachnoclostridium,NA
ASV_3,Bacteria,Firmicutes,Bacilli,Erysipelotrichales,Erysipelatoclostridiaceae,Erysipelatoclostridium,NA
ASV_4,Bacteria,Firmicutes,Clostridia,Oscillospirales,Ruminococcaceae,Ruminococcus,NA
ASV_5,Bacteria,Firmicutes,Clostridia,Lachnospirales,Lachnospiraceae,NA,NA
ASV_6,Bacteria,Firmicutes,Clostridia,Lachnospirales,Lachnospiraceae,NA,NA
ASV_7,Bacteria,Firmicutes,Clostridia,Lachnospirales,Lachnospiraceae,NA,NA
ASV_8,Bacteria,Firmicutes,Clostridia,Lachnospirales,Lachnospiraceae,NA,NA
ASV_9,Bacteria,Firmicutes,Bacilli,Lactobacillales,Lactobacillaceae,Lactobacillus,NA
ASV_10,Bacteria,Firmicutes,Clostridia,Lachnospirales,Lachnospiraceae,NA,NA
